In [1]:
import random
import numpy as np
from mido import Message, MidiFile, MidiTrack

In [2]:
#import a midi file
mid = MidiFile("midi/training/beethoven/elise.mid")

In [3]:
# print all events
for i, track in enumerate(mid.tracks):
    print('Track {}: {}'.format(i, track.name))
    for msg in track:
        #if msg.type == "note_on":
        print(msg)

Track 0: Für Elise
<meta message track_name name='Für Elise' time=0>
<meta message copyright text='Copyright © 2004 by Bernd Krueger' time=0>
<meta message text text='Ludwig van Beethoven' time=0>
<meta message text text='Poco moto' time=0>
<meta message text text='Fertiggestellt am 9.10.2004\n' time=0>
<meta message text text='Update am 30.8.2012\n' time=0>
<meta message text text='Dauer: 2:48 Minuten\n' time=0>
<meta message smpte_offset frame_rate=25 hours=32 minutes=0 seconds=3 frames=0 sub_frames=0 time=0>
<meta message time_signature numerator=3 denominator=8 clocks_per_click=12 notated_32nd_notes_per_beat=8 time=0>
<meta message key_signature key='C' time=0>
<meta message set_tempo tempo=867303 time=0>
<meta message set_tempo tempo=861203 time=460>
<meta message set_tempo tempo=887574 time=20>
<meta message set_tempo tempo=881187 time=40>
<meta message set_tempo tempo=874763 time=40>
<meta message set_tempo tempo=868558 time=40>
<meta message set_tempo tempo=862441 time=40>
<met

note_on channel=0 note=72 velocity=0 time=120
note_on channel=0 note=71 velocity=38 time=0
note_on channel=0 note=71 velocity=0 time=120
note_on channel=0 note=69 velocity=36 time=0
note_on channel=0 note=69 velocity=0 time=240
<meta message end_of_track time=0>
Track 2: Piano left
<meta message track_name name='Piano left' time=0>
program_change channel=0 program=0 time=0
control_change channel=0 control=7 value=100 time=0
control_change channel=0 control=10 value=64 time=0
note_on channel=0 note=45 velocity=37 time=1440
control_change channel=0 control=64 value=127 time=36
note_on channel=0 note=45 velocity=0 time=84
note_on channel=0 note=52 velocity=33 time=0
note_on channel=0 note=52 velocity=0 time=120
note_on channel=0 note=57 velocity=33 time=0
note_on channel=0 note=57 velocity=0 time=120
note_on channel=0 note=40 velocity=40 time=360
control_change channel=0 control=64 value=0 time=19
control_change channel=0 control=64 value=127 time=38
note_on channel=0 note=40 velocity=0 t

In [4]:
def get_total_beats(mid):
    max_ticks = 0
    for i, track in enumerate(mid.tracks):
        total_ticks = 0
        print('Track {}: {}'.format(i, track.name))
        for msg in track:
            if msg.type == "note_on" :
                total_ticks = total_ticks + msg.time
        if total_ticks> max_ticks:
           max_ticks = total_ticks 
        print("total ticks:" , total_ticks)
    print("max ticks: ", max_ticks)
    total_beats = int(max_ticks / mid.ticks_per_beat)
    return total_beats

In [5]:
len(mid.tracks)

8

In [6]:
### params #mid is is the midi file, 
### max_sim_notes is the maximum amount of simultaneous notes on a track, as of now most pieces have at most 4
### max_granularity is what is the minumum space that you can represent
### 1 is crotchets/quarters, 2 is quavers, 4 is semiquavers, etc. 4,or 8 is is a good default

### returns, tensor - the tensor that processed the whole midi only on the tracks that have note_on events
def convert_midi_to_tensor(mid, max_sim_notes, max_granularity):
    total_beats = get_total_beats(mid)
    tensor = np.zeros((len(mid.tracks), total_beats * max_granularity, max_sim_notes))
    note_on_dims = []
    for i, track in enumerate(mid.tracks):
            total_ticks = 0
            #print('Track {}: {}'.format(i, track.name))
            secondary_index = 0
            prev_index = -1
            note_on_in_track = False
            for msg in track:
                if msg.type == "note_on" :
                    note_on_in_track = True
                    total_ticks = total_ticks + msg.time
                    if msg.velocity > 0 :
                        #print(total_ticks * max_granularity / mid.ticks_per_beat)
                        curr_index = round(total_ticks * max_granularity / mid.ticks_per_beat)
                        #print(curr_index)
                        if prev_index == curr_index :
                            secondary_index = secondary_index + 1
                        else:
                            secondary_index = 0
                        prev_index = curr_index
                        tensor[i, curr_index, secondary_index] = msg.note
                        #print(curr_index, secondary_index)
            
            if note_on_in_track :
                note_on_dims.append(i)
    #print ("time series shape:", tensor.shape)
    return tensor[note_on_dims]

In [8]:
tensor = convert_midi_to_tensor(mid, 5, 4)
print(tensor.shape)

Track 0: Für Elise
total ticks: 0
Track 1: Piano right
total ticks: 90240
Track 2: Piano left
total ticks: 82946
Track 3: Beethoven Für Elise
total ticks: 0
Track 4: Copyright © 2004 by Bernd Krüger
total ticks: 0
Track 5: http://www.piano-midi.de
total ticks: 0
Track 6: Edition: 2012-08-30
total ticks: 0
Track 7: Spur 7
total ticks: 0
max ticks:  90240
(2, 752, 5)


In [9]:
### params #mid is is the midi file, 
### max_sim_notes is the maximum amount of simultaneous notes on all tracks, as of now most pieces have at most 6
### max_sim_notes_per_track is the maximum amount of simultaneous notes on a track, as of now most pieces have at most 4
### max_granularity is what is the minumum space that you can represent
### 1 is crotchets/quarters, 2 is quavers, 4 is semiquavers, etc. 4,or 8 is is a good default

### returns, time_series - the tensor thatrepresents the sorted notes on all tracks, per time step
def convert_midi_to_time_series(mid, max_sim_notes, max_sim_notes_per_track, max_granularity) :
    tensor = convert_midi_to_tensor(mid, max_sim_notes_per_track, max_granularity)
    all_tracks = np.concatenate(tensor[:], axis=1)
    all_tracks.sort(axis=1)
    concat_len = len(all_tracks[0])
    return  all_tracks[:,concat_len - max_sim_notes :concat_len]

In [11]:
time_series = convert_midi_to_time_series(mid, 5, 5, 8)
print(time_series.shape)

Track 0: Für Elise
total ticks: 0
Track 1: Piano right
total ticks: 90240
Track 2: Piano left
total ticks: 82946
Track 3: Beethoven Für Elise
total ticks: 0
Track 4: Copyright © 2004 by Bernd Krüger
total ticks: 0
Track 5: http://www.piano-midi.de
total ticks: 0
Track 6: Edition: 2012-08-30
total ticks: 0
Track 7: Spur 7
total ticks: 0
max ticks:  90240
(1504, 5)


In [12]:
print(time_series[1504-33:1504])
time_series.shape

[[ 0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0. 74.]
 [ 0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0. 72.]
 [ 0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0. 69.]
 [ 0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0. 60.]
 [ 0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0. 64.]
 [ 0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0. 69.]
 [ 0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0. 71.]
 [ 0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0. 64.]
 [ 0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0. 72.]
 [ 0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0. 71.]
 [ 0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0. 69.]
 [ 0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.]]


(1504, 5)

In [18]:
def get_training_data(time_series, record_size):
    result = np.zeros((len(time_series)+1 - record_size, time_series.shape[1], record_size))
    print(result.shape)
    idx = 0
    time_series_len = len(time_series)
    while idx <= time_series_len - record_size:
        result[idx] = time_series[idx:idx+record_size].T
        idx = idx + 1
    return result

In [19]:
training = get_training_data(time_series, 33)

(1472, 5, 33)


In [20]:
print(training[0])

[[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  0.  0.  0.  0. 45.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0. 76.  0. 75.  0. 76.  0. 75.  0. 76.  0.
  71.  0. 74.  0. 72.  0. 69. 52.  0. 57.  0.  0. 60.  0. 64.]]


In [ ]:
mid.length

In [ ]:
mid.ticks_per_beat

In [ ]:
dir(mid)

In [ ]:
#set to grand piano
#https://noterepeat.com/articles/how-to/213-midi-basics-common-terms-explained
mid.tracks[1][4].program = 0